# Chapter 8 Hands-on Exercise 2 - Exploring LangChain

In [1]:
# Install langchain
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


Setting up the environment variables to hold your variables

In [2]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "<Enter_API_Key_here>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://....openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-35"

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
            temperature=0,
            deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
            streaming=True,
)

Creating a prompt and storing it in messages, then sending the prompt to the LLM

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'finish_reason': 'stop'}, id='run-f0201fe1-04e1-4185-969a-0feeb6c7a2e3-0')

AIMessage from the cell above shows the raw output from the LLM. When in use with applications, we really only need the content from the LLM without the metadata. So we will use LangChain's parser to retrieve that field.

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [5]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

## Prompt Templates
LLM applications usually take user input and transforms it into a list of messages to pass to the LLM. Example of transformations in a chat scenario is to add the system message.

PromptTemplates are a concept in LanhChain that is designed to assist with transformations. They take in raw user input and return data in the form of a well-formed prompt that is ready to pass to the LLM.

As such, in this example, we will need to create a PromptTemplate that will take in two user variables:
language: The language to translate text into
text: The text to translate


In [6]:
from langchain_core.prompts import ChatPromptTemplate

#create string to hold the system message
system_template = "Translate the following into {language}:"

#create the PromptTemplate, which will be a combination of the system_message as well as a simpler template to hold the user message
prompt_template = ChatPromptTemplate.from_messages([("system", system_template), ("user", "{user_message}")])

In [7]:
#Testing the PromptTemplate
result = prompt_template.invoke({"language": "Italian", "user_message": "hi!"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Italian:'), HumanMessage(content='hi!')])

In [8]:
result.to_messages()

[SystemMessage(content='Translate the following into Italian:'),
 HumanMessage(content='hi!')]

Combine the prompt, model, and the output-parser into a chain and then invoke the chain

In [9]:
#Creating chain
chain = prompt_template | model | parser
#invoking chain
chain.invoke({"language": "italian", "user_message": "hi"})

'ciao'

# Serving with LangServe
Now that we know the chain works, we need to expose the chain. This is where we will use LangServe. LangServe helps developers expose LangChain chains as REST APIs.

Please RUN serve.py in WSL
>Python serve.py